# Community tourism

```{admonition} Disclaimer
:class: warning

This case study is a preliminary exploration intended only to illustrate a possible use case of DataBlox-OD. The insights presented here are derived from a limited subset of our GPS data and should not be considered conclusive of actual trends.

The views expressed here are those of the authors and do not necessarily reflect the views and policies of the Asian Development Bank (ADB) or its Board of Governors or the governments they represent.
```

Load the necessary libraries.

In [1]:
import os

import geopandas as gpd
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

import _community_tourism_util as util

pd.set_option("display.max_rows", 5)

%load_ext autoreload
%autoreload 2

Folder names

In [2]:
SAMPLE_DATA_DIRECTORY = os.path.join("..", "sample_data")
SAMPLE_OUTPUT_DIRECTORY = os.path.join("..", "sample_output")
CASE_STUDY_DATA_DIRECTORY = os.path.join("..", "case_study_data")
CASE_STUDY_OUTPUT_DIRECTORY = os.path.join("..", "case_study_output")

BOUNDARIES_DIRECTORY = os.path.join(SAMPLE_DATA_DIRECTORY, "boundaries")
TOURISM_DIRECTORY = os.path.join(SAMPLE_DATA_DIRECTORY, "tourism")
STAYPOINTS_DIRECTORY = os.path.join(SAMPLE_OUTPUT_DIRECTORY, "tourism", "staypoints")
ANNOTATED_TOURISM_TRIPS_DIRECTORY = os.path.join(
    SAMPLE_OUTPUT_DIRECTORY, "tourism", "annotated_tourism_trips"
)
COMMUNITY_TOURISM_TRIPS_DIRECTORY = os.path.join(
    CASE_STUDY_OUTPUT_DIRECTORY, "community_tourism_trips"
)
COMMUNITY_SAME_DAY_VISITS_DIRECTORY = os.path.join(
    CASE_STUDY_OUTPUT_DIRECTORY, "community_same_day_visits"
)
COMMUNITY_OVERNIGHT_VISITS_DIRECTORY = os.path.join(
    CASE_STUDY_OUTPUT_DIRECTORY, "community_overnight_visits"
)

os.makedirs(COMMUNITY_TOURISM_TRIPS_DIRECTORY, exist_ok=True)
os.makedirs(COMMUNITY_SAME_DAY_VISITS_DIRECTORY, exist_ok=True)
os.makedirs(COMMUNITY_OVERNIGHT_VISITS_DIRECTORY, exist_ok=True)

Administrative boundaries of Thailand

In [3]:
adm1 = gpd.read_file(os.path.join(BOUNDARIES_DIRECTORY, "thailand_adm1.geojson"))
adm3 = gpd.read_file(os.path.join(BOUNDARIES_DIRECTORY, "thailand_adm3.geojson"))

<hr>

## I. Profiling visitors to communities

We start by identifying visits to communities.

**Notes:** 
- Technically, by visits to communities, we mean visits to subdistricts that house communities (since we do not have the exact coordinates of the communities themselves). We refer to the list of communities from the Tourism Authority of Thailand.
- The scope of this case study is limited to domestic visitors from 2023 to 2024.

In [4]:
util.identify_tourism_trips_with_community_visits(
    ANNOTATED_TOURISM_TRIPS_DIRECTORY, COMMUNITY_TOURISM_TRIPS_DIRECTORY
)

Estimate the socioeconomic status of visitors based on the relative wealth index value of their usual residence. Developed by Meta, the reference [relative wealth index map](https://www.pnas.org/doi/10.1073/pnas.2113658119) has a spatial resolution of 2.4 km. 

In [5]:
visitor_profiles = util.get_relative_wealth_community_tourists(
    STAYPOINTS_DIRECTORY,
    os.path.join(CASE_STUDY_OUTPUT_DIRECTORY, "visitor_profiles.parquet"),
    ANNOTATED_TOURISM_TRIPS_DIRECTORY,
    COMMUNITY_TOURISM_TRIPS_DIRECTORY,
    os.path.join(CASE_STUDY_DATA_DIRECTORY, "tha_relative_wealth_index.csv"),
)

relative_wealth_statistics = (
    util.aggregate_community_tourist_relative_wealth_statistics(visitor_profiles)
)
print(
    f"Visitors who went to communities and belong to the {relative_wealth_statistics.index[0]}th percentile:",
    util.aggregate_community_tourist_relative_wealth_statistics(visitor_profiles).iloc[
        0
    ]["count"],
)

Visitors who went to communities and belong to the 95th percentile: 103


Compute the proportion of visitors who went to communities.

In [6]:
util.get_community_tourist_statistics(visitor_profiles)

Visitors: 2652
Visitors who went to communities: 141 (5.316742081447964%)


Identify where most visitors come from.

In [7]:
util.profile_provinces_of_residence_of_visitors(COMMUNITY_TOURISM_TRIPS_DIRECTORY)

  0%|          | 0/141 [00:00<?, ?it/s]

Bangkok: 31 (21.98581560283688%)
Chon Buri: 12 (8.51063829787234%)


`````{admonition} Insights
:class: tip
* Around 5% of domestic visitors went to communities.
* Most visitors to communities come from the wealthiest 5% based on the relative wealth of their usual residence.
* Most visitors to communities come from Bangkok (22%), followed by Chon Buri (9%).
`````

<hr>

## II. Profiling visits to communities

We start by investigating whether visits to communities are "standalone" visits or part of a multi-destination trip.

In [8]:
util.profile_num_destination_of_trips(COMMUNITY_TOURISM_TRIPS_DIRECTORY)

  0%|          | 0/141 [00:00<?, ?it/s]

Province
Mean: 1.8581560283687943
Median: 2.0
Min: 1
Max: 14
District
Mean: 2.8226950354609928
Median: 2.0
Min: 1
Max: 21


We can also distinguish between same-day visits (excursions) and overnight visits to communities.

In [9]:
util.identify_visits_to_communities(
    COMMUNITY_TOURISM_TRIPS_DIRECTORY,
    COMMUNITY_SAME_DAY_VISITS_DIRECTORY,
    same_day=True,
)

util.identify_visits_to_communities(
    COMMUNITY_TOURISM_TRIPS_DIRECTORY,
    COMMUNITY_OVERNIGHT_VISITS_DIRECTORY,
    same_day=False,
)

Profile the length of stay.

In [10]:
print("Same-day visits (minutes)")
util.profile_length_of_stay_in_communities(
    COMMUNITY_SAME_DAY_VISITS_DIRECTORY, same_day=True
)
print("===============")
print("Overnight visits (days)")
util.profile_length_of_stay_in_communities(
    COMMUNITY_OVERNIGHT_VISITS_DIRECTORY, same_day=False
)

Same-day visits (minutes)
Mean: 167.5693727870511
Median: 96.75
Min: 30.1
Max: 893.4833333333333
Overnight visits (days)
Mean: 1.2119565217391304
Median: 1.0
Min: 1
Max: 5


Profile the distance traveled by visitors from their residence to the communities.

In [11]:
print("Same-day visits (km)")
distances_same_day_visits = util.profile_distance_from_residence_to_communities(
    COMMUNITY_SAME_DAY_VISITS_DIRECTORY,
    os.path.join(CASE_STUDY_OUTPUT_DIRECTORY, "visitor_profiles.parquet"),
)
print("===============")
print("Overnight visits (km)")
distances_overnight_visits = util.profile_distance_from_residence_to_communities(
    COMMUNITY_OVERNIGHT_VISITS_DIRECTORY,
    os.path.join(CASE_STUDY_OUTPUT_DIRECTORY, "visitor_profiles.parquet"),
)

Same-day visits (km)
Mean: 201.7460207347775
Overnight visits (km)
Mean: 111.5534906093826


In fact, we find a statistical difference between the distance traveled by same-day visitors and the distance traveled by overnight visitors.

In [12]:
stats.mannwhitneyu(
    distances_same_day_visits, distances_overnight_visits, alternative="greater"
)

MannwhitneyuResult(statistic=np.float64(80099.0), pvalue=np.float64(1.3023906259737814e-11))

`````{admonition} Insights
:class: tip
* Visits to communities are usually part of a longer trip that involves going to 2 provinces or 3 districts on average.
* Same-day visitors stay in communities for an average of 3 hours. Overnight visitors stay for up to 5 nights, but with an average stay of only 1 night.
* Same-day visitors tend to travel longer distances from their residence to the communities. On average, they travel 202 km, while overnight visitors travel 112 km. (To put this distance into perspective, Bangkok to Lop Buri is roughly 117 km.)
`````

<hr>

## III. Profiling communities

We start by counting the number of visitors per community.

In [13]:
communities_and_visitors = util.identify_communities_and_visitors(
    COMMUNITY_TOURISM_TRIPS_DIRECTORY
)
visitor_counts_df = util.get_num_visitors_per_community(communities_and_visitors)
visitor_counts_df

  0%|          | 0/141 [00:00<?, ?it/s]

,community,num_visitors
0,Ban Wang Som Sa Community,17
1,Chanthaboon Waterfront Community,16
...,...,...
25,Koh Phithak Community,1
26,Ban Nam Chieo Community,1


Estimate the accessibility of communities based on the mean motorized friction surface of their respective subdistricts. The motorized friction surface is a measure of how "hard" it is to cross a given area by motorized transportation (as opposed to walking only). To this end, we refer to the [motorized friction surface map](https://www.nature.com/articles/s41591-020-1059-1) from Malaria Atlas, which has a spatial resolution of 1 km. 

We also create a binary indicator of accessibility based on the median of the motorized friction surface values across all the subdistricts of Thailand.

In [14]:
accessibility_df = util.profile_communities(
    adm3,
    list(communities_and_visitors.keys()),
    os.path.join(TOURISM_DIRECTORY, "thailand_communities.csv"),
    os.path.join(CASE_STUDY_DATA_DIRECTORY, "motorized_friction_surface.geotiff"),
    os.path.join(CASE_STUDY_OUTPUT_DIRECTORY, "zonalstats_accessibility.parquet"),
    "accessibility",
    is_high_covariate_below_threshold=True,
)

accessibility_df["high_accessibility"].value_counts()

high_accessibility
False    17
True     10
Name: count, dtype: int64

Estimate the electrification of communities based on the average intensity of nighttime lights of their respective subdistricts. To this end, we refer to the [nighttime lights imagery](https://www.earthdata.nasa.gov/topics/human-dimensions/nighttime-lights) from NASA, which has a spatial resolution of 15 arc-seconds (around 450 m at the equator). 

We also create a binary indicator of electrification based on the 80<sup>th</sup> percentile of the nighttime light intensities across all the subdistricts of Thailand.

In [15]:
electrification_df = util.profile_communities(
    adm3,
    list(communities_and_visitors.keys()),
    os.path.join(TOURISM_DIRECTORY, "thailand_communities.csv"),
    os.path.join(CASE_STUDY_DATA_DIRECTORY, "tha_nighttime_lights.tif"),
    os.path.join(CASE_STUDY_OUTPUT_DIRECTORY, "zonalstats_electrification.parquet"),
    "electrification",
    threshold=0.80,
)

electrification_df["high_electrification"].value_counts()

high_electrification
False    16
True     11
Name: count, dtype: int64

Estimate the biodiversity intactness of communities based on the average abundance of originally-present species in the communities' respective subdistricts. To this end, we refer to the [biodiversity intactness index map](https://geohub.data.undp.org/data/683138801529c7aa73636710d4b55927) from the United Nations Environment Programme, which has a spatial resolution of 1 km. 

We also create a binary indicator of biodiversity intactness based on the 80<sup>th</sup> percentile of the biodiversity intactness values across all the subdistricts of Thailand.

In [16]:
biodiversity_df = util.profile_communities(
    adm3,
    list(communities_and_visitors.keys()),
    os.path.join(TOURISM_DIRECTORY, "thailand_communities.csv"),
    os.path.join(CASE_STUDY_DATA_DIRECTORY, "Biodiversity_Intact_Index.tif"),
    os.path.join(
        CASE_STUDY_OUTPUT_DIRECTORY, "zonalstats_biodiversity_intactness.parquet"
    ),
    "biodiversity_intactness",
    threshold=0.80,
)

biodiversity_df["high_biodiversity_intactness"].value_counts()

high_biodiversity_intactness
False    17
True     10
Name: count, dtype: int64

Compute the population densities of the subdistricts as well. To this end, we refer to the gridded population map from the [Global Human Settlement Layer](https://www.tandfonline.com/doi/full/10.1080/17538947.2024.2390454), which has a spatial resolution of 100 m.

We also create a binary indicator of population density based on the median of the population densities across all the subdistricts of Thailand.

In [17]:
population_density_df = util.compute_population_density(
    adm3,
    list(communities_and_visitors.keys()),
    os.path.join(TOURISM_DIRECTORY, "thailand_communities.csv"),
    os.path.join(
        CASE_STUDY_DATA_DIRECTORY, "GHS_POP_E2025_GLOBE_R2023A_54009_100_V1_0.tif"
    ),
    temp_paths=[
        os.path.join(CASE_STUDY_OUTPUT_DIRECTORY, "zonalstats_population.parquet"),
        os.path.join(CASE_STUDY_OUTPUT_DIRECTORY, "zonalstats_area.parquet"),
    ],
)

population_density_df["high_population_density"].value_counts()

high_population_density
True     15
False    12
Name: count, dtype: int64

Construct a cross-sectional dataset where each observation corresponds to the number of visitors to a community, along with the community's profile based on the attributes we computed earlier.

In [18]:
community_profile_df = pd.merge(
    pd.merge(
        pd.merge(pd.merge(visitor_counts_df, accessibility_df), electrification_df),
        biodiversity_df,
    ),
    population_density_df,
)
community_profile_df[
    [
        "community",
        "num_visitors",
        "high_accessibility",
        "high_electrification",
        "high_biodiversity_intactness",
        "high_population_density",
    ]
]

,community,num_visitors,high_accessibility,high_electrification,high_biodiversity_intactness,high_population_density
0,Ban Wang Som Sa Community,17,True,True,False,True
1,Chanthaboon Waterfront Community,16,True,True,True,True
...,...,...,...,...,...,...
25,Koh Phithak Community,1,False,False,True,False
26,Ban Nam Chieo Community,1,False,False,True,True


To investigate how accessibility and electrification impact the attractiveness of a community as a tourist destination, we regress the number of tourists as a function of accessibility and electrification. We also include the population density as a covariate.

In [19]:
X = sm.add_constant(
    community_profile_df[
        [
            "high_accessibility",
            "high_electrification",
            "high_biodiversity_intactness",
            "high_population_density",
        ]
    ].astype(int)
)

y = np.log(community_profile_df["num_visitors"])

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           num_visitors   R-squared:                       0.439
Model:                            OLS   Adj. R-squared:                  0.337
Method:                 Least Squares   F-statistic:                     4.311
Date:                Tue, 26 Aug 2025   Prob (F-statistic):             0.0100
Time:                        18:11:46   Log-Likelihood:                -28.459
No. Observations:                  27   AIC:                             66.92
Df Residuals:                      22   BIC:                             73.40
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            1.0388      0.299      3.474      0.002       0.419       1.659
high_accessibility               1.1912      0.443      2.687      0.013       0.272       2.111
high_electrification             1.1500      0.460      2.502      0.020       0.197       2.103
high_biodiversity_intactness     0.3737      0.343      1.088      0.288      -0.338       1.086
high_population_density         -1.3813      0.498     -2.776      0.011      -2.413      -0.349
==============================================================================
Omnibus:                        0.348   Durbin-Watson:                   1.255
Prob(Omnibus):                  0.840   Jarque-Bera (JB):                0.496
Skew:                          -0.205   Prob(JB):                        0.780
Kurtosis:                       2.478   Cond. No.                         5.79
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Since there is no evidence of [spatial autocorrelation](https://www.paulamoraga.com/book-spatial/spatial-autocorrelation.html), the results from our ordinary least squares regression hold.

In [20]:
util.is_there_spatial_autocorrelation(
    model,
    community_profile_df,
    adm3,
    os.path.join(TOURISM_DIRECTORY, "thailand_communities.csv"),
)

Moran's I: 0.06900470931359154
p-value: 0.177


`````{admonition} Insights
:class: tip
* 63% of the visited communities have below-median accessibility levels.
* Enhancing accessibility and electrification benefits community tourism, with our preliminary analysis showing that:
    * Communities with accessibility levels above the median see a 3.29× increase in the number of visitors.
    * Communities with electrification levels above the 80<sup>th</sup> percentile see a 3.16× increase in the number of visitors.
`````

<hr>

## IV. Developing community tourism corridors

Focusing on visitors to Ban Wang Som Sa Community (the most visited community in our dataset), we identify other places in Phitsanulok province that they may have also visited. Knowing these locations provides a valuable starting point for developing community tourism corridors.

In the visualization below, we focus on the following:
- **Main tourist attractions** (from the [website of the Tourism Authority of Thailand](https://www.tourismthailand.org/Destinations)). We classify them into cultural (blue pins with a landmark icon), service (red pins with a store icon), entertainment (blue-green pins with a ticket icon), and nature (green pins with a tree icon) attractions.
- **Transportation hubs** (from the [Humanitarian OpenStreetMap Team](https://data.humdata.org/organization/hot)). We classify them into sea ports (purple pins with a ship icon), airports (purple pins with a plane icon), and railways (purple pins with a train icon).
- **Accommodation** (from the [Humanitarian OpenStreetMap Team](https://data.humdata.org/organization/hot)). They are marked by pink pins with a bed icon.
- **Food and beverage establishments** (from the [Humanitarian OpenStreetMap Team](https://data.humdata.org/organization/hot)). They are marked by beige pins with a utensils icon.

The community of interest is marked by a black pin with a people icon.

In [21]:
places = util.identify_places_visited_with_community(
    "Ban Wang Som Sa Community",
    COMMUNITY_TOURISM_TRIPS_DIRECTORY,
    communities_and_visitors,
    os.path.join(TOURISM_DIRECTORY, "thailand_tourist_attractions.parquet"),
    os.path.join(TOURISM_DIRECTORY, "thailand_communities.csv"),
    os.path.join(
        TOURISM_DIRECTORY,
        "openstreetmap",
        "hotosm_tha_sea_ports_polygons_geojson.geojson",
    ),
    os.path.join(
        TOURISM_DIRECTORY,
        "openstreetmap",
        "hotosm_tha_airports_polygons_geojson.geojson",
    ),
    os.path.join(
        TOURISM_DIRECTORY, "openstreetmap", "hotosm_tha_railways_lines_geojson.geojson"
    ),
    os.path.join(
        TOURISM_DIRECTORY,
        "openstreetmap",
        "hotosm_tha_points_of_interest_polygons_geojson.geojson",
    ),
    adm1,
    adm3,
)
places

,place,category,geometry
0,Textile Museum At Naresuan University,Cultural,POINT (100.19348 16.751)
1,Thai Bird Garden,Nature,POINT (100.26818 16.80566)
...,...,...,...
809,ศูนย์อาหาร,food_and_beverages,"POLYGON ((100.1931 16.74431, 100.19312 16.7442..."
810,Ranna,food_and_beverages,"POLYGON ((100.19766 16.75091, 100.19769 16.750..."


In [22]:
util.plot_visited_places(places)

This same exercise can be done for any community of interest.